In [ ]:
import tensorflow as tf

In [ ]:
# TODO make tensorflow dataset
# does this require the arrays to be pre-processed?

def make_tf_dataset(all_frags):
    all_arrays = [arr for pdb_arrs in all_frags.values() for arr in pdb_arrs]
    features = [''.join(arr.tolist()[2]) for arr in all_arrays] # Residue sequences
    labels = [arr[1,0] for arr in all_arrays] # Cluster IDs
    
    features = tf.keras.preprocessing.text.one_hot()

    print(labels)
    print(features)

    # dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    # dataset = dataset.shuffle(buffer_size=len(features))
    # dataset = dataset.batch(32)
    # dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)